## DWF Searching candidates algorithm for DWF source matching with existing catalogs
### Chris M. Murphy 
### A. Cucchiara

* Jan 22: Added the "try / else" sections: this to prevent the code to crash in case objects are not in the catalogue searches
* Separated the section that calculates the separation in a different function
* Added a "write all" function that writes all the files one by one if tables exist
* Feb 04: Added COSMOS, CFHTLS searches. Added angular separation(arcmin) and transverse separation (kpc) from COSMOS galaxy

In [19]:
#import statements
import numpy as np
from astropy.table import Table, Column
from astropy.io import ascii
from astroquery.vizier import Vizier
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.cosmology import WMAP9 as cosmo

In [2]:
def calcsep(ra1,dec1,ra2,dec2):
    #ra1, dec1 are the transient candidates position 
    #ra2, dec2 are the catalogue sources
    candidate = SkyCoord(ra1, dec1, frame='fk5', unit='deg')
    result_near_candidate = SkyCoord(ra2, dec2,frame='fk5', unit='deg')
    sep = candidate.separation(result_near_candidate)
    return sep

In [40]:
#Search function
def Search(RA, DEC, Search_Radius, catalog):
        v = Vizier(columns=["**"])
        #2MASS Search
        if catalog == "2MASS":
            try:
                result = Vizier.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                             unit=(u.deg, u.deg),
                                             frame='icrs'),
                                             width=Search_Radius,
                                             catalog=['II/246/out'])
                #r is a table of all results found from 2MASS Search
                r = result['II/246/out']
                
                #Create empty of separations array to fill
                separray=[] 
 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)
                c = Column(data=separray, name='Sep_as')        
                #Make a table of arrays
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']), np.array(r['Jmag'])\
                                     , np.array(r['e_Jmag']), np.array(r['Hmag']), np.array(r['e_Hmag']),\
                                     np.array(r['Kmag']), np.array(r['e_Kmag'])]\
                                     ,names=('RA','DEC','Jmag','Jmag_error','Hmag','Hmag_error',\
                               'Kmag', 'Kmag_error'))
                resultTable.add_column(c)
            except (RuntimeError, TypeError, NameError):
                print("No object in the 2MASS catalog")
                resultTable=[]
                
            return resultTable
        #2MASSX Search 
        if catalog == '2MASSX':
            try:
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=['VII/233'])

                r = result['VII/233/xsc']

                #Create empty of separations array to fill
                separray=[] 
    
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)        
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']),\
                                np.array(r['Kb_a']),
                                np.array(r['r.fe']), np.array(r['J.ext']), np.array(r['e_J.ext']),
                                np.array(r['H.ext']),np.array(r['e_H.ext']), np.array(r['K.ext']),\
                                 np.array(r['e_K.ext']),np.array(r['xid'])],
                                names = ('RA', 'DEC', 'Kb_a', 'r.fe', 'J.ext', 'e_J.ext', \
                                         'H.ext','e_H.ext', 'K.ext','e_K.ext','xid'))
                resultTable.add_column(c)
                
            except (RuntimeError, TypeError, NameError):
                print("No object in the 2MASSX catalog")
                resultTable=[]

            return resultTable
        #USNOB1 Search
        if catalog == "USNOB1":
            try:
                result = Vizier.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                             unit=(u.deg, u.deg),
                                             frame='icrs'),
                                             width=Search_Radius,
                                             catalog=['I/284/out'])
                #r is a table of all results found from USNOB1 search
                r = result['I/284/out']
            
                #Create empty of separations array to fill
                separray=[] 
 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)
                c = Column(data=separray, name='Sep_as')
            
                #Create a table containing desired columns of r and the seperation
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']), \
                                 np.array(r['B1mag']), np.array(r['R1mag']), np.array(r['B2mag'])\
                                 , np.array(r['R2mag']),np.array(r['Imag'])], 
                        names = ('RA', 'DEC', 'B1mag', 'R1mag','B2mag', 'R2mag', 'Imag'))
                resultTable.add_column(c)
            except (RuntimeError, TypeError, NameError):
                print("No object in the USNOB1 catalog")
                resultTable=[]

            return resultTable
        
        #PAN-STARRS Search
        if catalog == 'PS1':
            try:
                result = Vizier.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                             unit=(u.deg, u.deg),
                                             frame='icrs'),
                                             width=Search_Radius,
                                             catalog=['II/349/ps1'])
                #r is a table of all results found from PAN-STARRS search
                r = result['II/349/ps1']
            
                #Create empty array to fill
                separray=[] 
 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)
                c = Column(data=separray, name='Sep_as')

                #Make a table of arrays
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']),np.array(r['objID']),\
                                 np.array(r['gmag']), np.array(r['e_gmag']), np.array(r['rmag']),\
                                 np.array(r['e_rmag']), np.array(r['imag']),\
                                 np.array(r['e_imag']), np.array(r['zmag']), np.array(r['e_zmag']), \
                                 np.array(r['ymag']),np.array(r['e_ymag'])],\
                                    names=('RA', 'DEC', 'ID', 'Gmag', 'Gmag_error', 'Rmag', \
                                           'Rmag_error', 'Imag', 'Imag_error','Zmag', 'Zmag_error',\
                                           'Ymag', 'Ymag_error'))
                resultTable.add_column(c)
            except (RuntimeError, TypeError, NameError):
                print("No object in the PS1 catalog")
                resultTable=[]

            return resultTable
        #COSMOS
        if catalog == 'COSMOS':
            try:
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=["II/284"])

                r = result['II/284/cosmos']
            
                #Create empty array to fill
                separray=[] 

                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)            
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['COSMOS']), np.array(r['RAJ2000']), np.array(r['DEJ2000']), 
                                np.array(r['istar']), np.array(r['imagA']), np.array(r['n_imagA']), 
                                np.array(r['Bmag']), np.array(r['Vmag']),
                                np.array(r['gmag']), np.array(r['rmag']), np.array(r['zmag']), np.array(r['NB816']),
                                np.array(r['zphot']), np.array(r['F814W']), np.array(r['e_F814W'])], 
                           names = ('COSMOS', 'RAJ2000', 'DEJ2000', 'istar', 'imagA', 'n_imagA', 'Bmag', 
                                    'Vmag', 'gmag', 'rmag', 'zmag', 'NB816', 'zphot', 
                                    'F814W', 'e_F814W'))
                resultTable.add_column(c)
                phys_sep=Column(data=cosmo.kpc_proper_per_arcmin(resultTable['zphot']).value/60.*c, name='Sep_kpc')
                resultTable.add_column(phys_sep)
            except (RuntimeError, TypeError, NameError):
                print("No object in the COSMOS catalog")
                resultTable=[]

            return resultTable
    
        #COSMOS XMMNEWTON
        if catalog == 'COSMOS-XMMN':
            try: 
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=["J/A+A/497/635"])

                r = result['J/A+A/497/635/catalog']

                #Create empty array to fill
                separray=[] 

                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)             
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']), np.array(r['ePos']),
                                 np.array(r['XID']), np.array(r['XMMU']), np.array(r['S.5-2']), 
                                 np.array(r['L.5-2']), np.array(r['S2-10']), np.array(r['L2-10']), 
                                 np.array(r['S5-10']), np.array(r['L5-10'])], 
                    names = ('RA', 'DEC', 'ePos', 'XID', 'XMMU', 'S.5-2', 'L.5-2', 
                             'S2-10', 'L2-10', 'S5-10', 'L5-10'))
                resultTable.add_column(c)
            except (RuntimeError, TypeError, NameError):
                print("No object in the COSMOS-XMMNewton catalog")
                resultTable=[]

            return resultTable
    
        #Morphology
        if catalog == 'COSMOS-Morph':
            try:
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=["VII/265"])

                r = result['VII/265/morpho']
            
                #Create empty array to fill
                separray=[] 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)              
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['ID']), np.array(r['RAJ2000']), np.array(r['DEJ2000']), np.array(r['Imag']),
                                 np.array(r['Mph1']), np.array(r['Mph2']), np.array(r['Mph3'])], 
                           names = ('ID', 'RA', 'DEC', 'Imag', 'Mph1', 'Mph2', 'Mph3'))
                resultTable.add_column(c)        
            except (RuntimeError, TypeError, NameError):
                print("No object in the COSMOS-Morphology catalog")
                resultTable=[]

            return resultTable
    
        #COSMOSVLA 
        if catalog == 'COSMOSVLA':
            try: 
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=["J/ApJS/188/384"])

                r = result['J/ApJS/188/384/table3']
                #Create empty array to fill
                separray=[] 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond)              
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']), np.array(r['COSMOSVLADP']), 
                                 np.array(r['Sp_1.4_']), np.array(r['Sp_1.4_c']), np.array(r['Si_1.4_']),
                                 np.array(r['e_Si_1.4_']), np.array(r['rmsbg']),np.array(r['Bmaj']),
                                 np.array(r['Bmin']), np.array(r['R']), np.array(r['Det'])], 
                           names = ('RA', 'DEC', 'COSMOSVLADP', 'Sp_1.4_', 'Sp_1.4_c', 'Si_1.4_', 'e_Si_1.4_',
                                    'rsmbg', 'Bmaj','Bmin', 'R', 'Det'))
                resultTable.add_column(c)        

            except (RuntimeError, TypeError, NameError):
                print("No object in the COSMOSVLA catalog")
                resultTable=[]

            return resultTable        
    
    
        #CFHTLS Deep Search
        #II/317/cfhtls_d
        if catalog == 'CFHTLS_d':
            try:
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=['II/317'])
                r = result['II/317/cfhtls_d']
                
                #Create empty array to fill
                separray=[] 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond) 
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']), np.array(r['gmagA']), 
                                 np.array(r['e_gmagA']), np.array(r['rmagA']), np.array(r['e_rmagA']),
                                 np.array(r['imagA']), np.array(r['e_imagA']), np.array(r['ymagA']),
                                 np.array(r['e_ymagA']), np.array(r['zmagA']), np.array(r['e_zmagA'])],
                            names = ('RA', 'DEC', 'gmagA', 'e_gmagA', 'rmagA', 'e_rmagA', 'imagA', 'e_imagA', 
                                     'ymagA', 'e_ymagA','zmagA', 'e_zmagA'))
                resultTable.add_column(c)        
            except (RuntimeError, TypeError, NameError):
                print("No object in the CFHTLS_d catalog")
                resultTable=[]

            return resultTable

        #CFHTLS Wide Search
        #II/317/cfhtls_w
        if catalog == 'CFHTLS_w':
            try:
                result = v.query_region(coord.SkyCoord(ra= RA, dec= DEC,
                                                         unit=(u.deg, u.deg),
                                                         frame='icrs'),
                                                         width=Search_Radius,
                                                         catalog=['II/317'])
                r = result['II/317/cfhtls_w']
                
                #Create empty array to fill
                separray=[] 
                #Calcule the seperation between the input source and the output sources
                for x in range(0, len(r['RAJ2000'])):   
                    separray.append(calcsep(RA,DEC,r['RAJ2000'][x], r['DEJ2000'][x]).arcsecond) 
                c = Column(data=separray, name='Sep_as')
                resultTable = Table([np.array(r['RAJ2000']), np.array(r['DEJ2000']), np.array(r['gmagA']), 
                                 np.array(r['e_gmagA']), np.array(r['rmagA']), np.array(r['e_rmagA']),
                                 np.array(r['imagA']), np.array(r['e_imagA']),
                                np.array(r['zmagA']), np.array(r['e_zmagA'])],
                            names = ('RA', 'DEC', 'gmagA', 'e_gmagA', 'rmagA', 'e_rmagA', 'imagA', 'e_imagA', 
                                     'zmagA', 'e_zmagA'))
                resultTable.add_column(c)        
            except (RuntimeError, TypeError, NameError):
                print("No object in the CFHTLS_w catalog")
                resultTable=[]

            return resultTable

In [3]:
def WriteFile(ID, catalog, table):
    filename = 'DWF'+str(ID)+'_'+catalog+'.txt'
    file = ascii.write(table, filename, overwrite=True)
    return file

In [5]:
#Q = Search(RA, DEC, Search_Radius, catalog)
#print(Q)

In [39]:
def searchAll(ra,dec,sep,ID):
    cats = {}
    keys = ['2MASS','2MASSX','USNOB1','PS1','COSMOS','COSMOS-XMMN','COSMOS-Morph','COSMOSVLA','CFHTLS_d','CFHTLS_w']
    for i in range(len(keys)):
        cats[keys[i]]=Search(ra,dec,sep,keys[i])
    return cats

In [5]:
def writeall(ID, matches):
    k=matches.keys()
    for i in range(len(k)):
        #print len(matches[k[i]])
        if len(matches[k[i]]) !=0:
            filename = 'DWF'+str(ID)+'_'+str(k[i])+'.txt'
            ascii.write(matches[k[i]], filename, overwrite=True)
    

In [43]:
S=Search(150.49813,+01.998845, '10s',"COSMOS")
print S

 COSMOS  RAJ2000  DEJ2000 istar ... e_F814W      Sep_as        Sep_kpc   
------- --------- ------- ----- ... ------- --------------- -------------
1187758 150.49813 1.99883  0.14 ...  0.6619 0.0540000000003 0.46611746616


In [10]:
tabs=searchAll(104.345,+24.56, '20s','340')

No object in the 2MASSX catalog


In [13]:
writeall('345',tabs)